---
>「不用意にもらす言葉こそ、ほんとうらしいものをふくんでいるのだ。 」
>
> 太宰治
---

# LangChain
LangChainは日々更新されている
- これは、この授業テキスト全般に言えることであるが、特に後半は更新が頻繁に行われている
- アップデートにより実行できない場合もあるが、その場合は速やかに申し出ること

その前に、OpenAPIのChat APIについて学ぶ

なお、このノートブックは、GPUを使わないCPUランタイムを利用している

*** 注意 ***

ChatGPTは大人気のサービスのため負荷が集中しており、無償利用枠がかなり少なく、期限切れや、無料利用料金枠切れ、さらには、1分あたり利用は3回までという厳しい制限が課せられている

無料料金枠などの問題は再度アカウントを取得すればよいが、1分あたり利用は3回までという制限はかなり厳しい

例えば、次のような文章を含むエラーが出力された場合は、しばらく待って再度実行する必要がある

```
WARNING:RateLimitError: Rate limit reached for default-gpt-3.5-turbo on requests per min. Limit: 3 / min. Please try again in 20s.
```

したがって、無償枠の場合、このノートブックを纏めて全て実行とするとエラーになるため注意すること

もし、まとめて実行する場合、途中で実行を待ってスロットを使いつくさないようにする必要があるため、次の設定を行うとよい

In [ ]:
import time
openai_wait = True

# Chat APIを利用する


## API Keyの発行

openai.comにアクセスし、USERメニュにあるAPI keysでAPI keyを発行する
- セキュリティのため、プロジェクトごとに異なるキーを利用すること
- ここでは、dataai-keyという鍵をつくるとよい
- 発行された鍵をコピーしておくこと

## API Keyを使えるようにする

発行したKeyを次のコードにペーストして利用する

In [ ]:
!pip install python-dotenv

In [ ]:
!echo "OPENAI_API_KEY=発行したAPI Keyをここに張り付けること" > .env

In [ ]:
from dotenv import load_dotenv
load_dotenv(verbose=True)

True

APIはOpenAI Chat APIに統一された

モデルは、2023年夏時点で最新かつ無料で利用できる、gpt-3.5-turboを利用する

## APIを使う

まず、ChatGPTに、挨拶して、ちゃんとAPIが使えているかどうかを確認する

"Hello. Am I using the API correctly?"

と聞いて回答を実際に得る

必要なライブラリは、単純に次の2つ
- インターネットを利用して、シンプルにRESTでリクエストを投げてレスポンスを得るためのrequests
- JSONフォーマットを扱うためのjson

In [ ]:
import requests
import json

In [ ]:
url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + os.environ["OPENAI_API_KEY"]
}
data = {
    "model": "gpt-3.5-turbo",
    "messages": [
        {"role": "user", "content": "Hello. Am I using the API correctly?"}
    ],
    "temperature": 0,
}

response = requests.post(url=url, headers=headers, json=data)
print(json.dumps(response.json(), indent=2))

{
  "id": "chatcmpl-7yzDI7g6SSeBubXU97KPgQ7zAhkK6",
  "object": "chat.completion",
  "created": 1694769340,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! I'm an AI language model and I'm here to help you. Could you please provide more details about which API you are referring to and what specific issue or question you have?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 16,
    "completion_tokens": 38,
    "total_tokens": 54
  }
}


返事として、"content"にあるように、

"Hello! I'm an AI language model and I'm here to help you. Could you please provide more details about the API you are using and what you are trying to achieve?"

と返ってきた

「こんにちは！私はAI言語モデルです。使用されているAPIの詳細と、何を達成しようとしているのかを教えていただけますか？」

だそうだ

なお、パラメータについて、
- temperatureは0に近いほど、同じ回答を出力するようになる
- max_tokensは返答として最大何文字返すかを指定する
  - ChatGPTのAPIは、利用文字数(トークン数)により課金されるため、コスト低減を考えるのであれば重要である  
  デフォルトは16であるが、これは通常の利用ではかなり少ないといえる
- nは同一質問に対する返答数を指定する  
  この場合temparatureを大きめの値にしなければ、同じ回答が並ぶことになる

ひとまず使えるようになったであろう

### エラートラブル(1)

次のように表示される場合は、APIを利用するための無償枠を既に使い切ったか、時間が過ぎたためExpireしたことを意味する

```
{
  "error": {
    "message": "You exceeded your current quota, please check your plan and billing details.",
    "type": "insufficient_quota",
    "param": null,
    "code": "insufficient_quota"
  }
}
```

- 無償で続けたい場合は、新しいアカウントをつくるとよい
- もちろん、コストを支払ってもよい

新しいアカウントをつくる

- ただし、メールアドレスが必要となる
  - 既にメールアドレスが枯渇している場合は、フリーメールアドレスを取得するとよい

- ブラウザのシークレットモードで、openai.comを開く(クッキーで既に持っているアカウント情報を利用しないようにするため)

- 右上のメニューからLoginを選択し、Sign upを選択する
  - メールアドレスとパスワードを入力する
  - 確認メールが届くのでVerifyする

- OpenAIのページに行くと、名前や誕生日の入力が求められる

- スマートフォンの番号を入れて、コードを受け取る
  - 利用済の番号でも問題ない

- APIをクリックして、OpenAI platformに行き、右上の丸いアイコンをクリックして個人メニューに入る

- 左のタブでAPI Keysを選択する

- "Create new secret key"を選択し、dataai-keyという鍵を作成してコピーしておく

以上、あらたに入手した鍵を利用して再実行すること
- ただし5ドル分しかないので注意すること

### エラートラブル(2)

頻繁にアクセスすると、エラーが発生する

これは、単位時間あたりのアクセス数が制限されているためである
- しばらく待ってリトライすること

### roleについて

"role"は、次の3つがある
- userはChatGPTのユーザで、皆さんのこと
- assistantがChatGPTによる回答を指し、ChatGPTのこと
- systemはassistantのふるまいを制御するために利用

但し、gpt-3.5-turboでは、systemは利用しない


## 重要なこと

### ブラウザとの違い

APIを利用するのと、ブラウザを利用して利用する場合と最も異なるのは、APIの利用では過去の会話のやり取りを一切考慮しないという点である

したがって、APIを利用する場合で過去の会話を参照したい場合は、過去の会話そのものを全てmessageに記載する必要がある

### 料金を確認すること

https://openai.com/pricing にアクセスすると、

| Model	| Input	| Output |
|:---|:---|:---|
| GPT-3.5-Turbo 4K context | \$0.0015 / 1K tokens | \$0.002 / 1K tokens |

と記載されている

現在いくらつかったかは、

https://platform.openai.com/account/usage

にアクセスして確認するとよい

### トークン数

トークンは基本的に単語数であるが、ChatGPTを含む多くのLMにおいて、膨大な単語を効率よく学習するため、一つの単語を複数のトークンに分割するということが良く行われる
- 例えば、"humburger"は、"hum", "bur", "ger"の3つに分解される

また、日本語と英語ではトークン数のカウント方法が異なる
- 日本語は内部で英語に変換されて処理されているため

- 日本語は1トークンはおよそ4文字、英語では主疎0.75語に相当する
  - 一般に直接英語を利用した方がお得といわれている
  - 実際2倍程度の開きがある

このトークン数は、各モデルの入力や出力サイズの制限にも利用される

直接文字数を計数したい場合は、https://platform.openai.com/tokenizer にアクセスして、文章を入力するとよい




# GPT APIを用いたアプリケーション実装

料理名を入力することで、材料と手順、調理時間、お勧めのサイドメニューなどを表示するというアプリを作成する

## 全体の構成

### APIまでの通信手順

以下の手順を踏む
- スマートフォンやPC、Webのアプリを利用して、レシピ名をサーバプログラムに送信する
- サーバプログラムは、プロンプトエンジニアリングを行い、APIキーを付与してOpenAIのAPIを叩く

レシピ生成アプリが直接OpenAI APIを叩くような構成は、APIキーが漏えいするため普通は行わない

<img src="http://class.west.sd.keio.ac.jp/dataai/text/chatapi1.jpg" width=700>

# LangChain

## LangChainとは？

LLMを使ったアプリケーション開発フレームワーク
- PythonとJavaScript/TypeScriptの2つがある
- フリーで利用できる

詳細は公式のドキュメントを参照すること
- 過去のバージョンのマニュアルを見る場合は、githubにあるlangchainに行き、docsのreleasesから目的のバージョンを選択するとよい

## Module

LangChainにはmoduleと呼ばれる要素があり、Models, Prompts, Chains, Indexes, Memory, Agentsである



### Models module

LongChainで利用する機械学習モデルのこと
- Chat Models: Open AIのChatAPIのためのモジュール
- Text Embedding Models: テキストをベクトル化するモデル



先に示した簡単な対話プログラムを再構成する
- 先ほどよりもシンプルに記述できることがわかるであろう
- 内部でjsonに変換され通信が行なわれている

In [ ]:
!pip install --quiet langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.4 MB/s eta 0:00:00


In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

result = llm.predict("自己紹介してください。")
print(result)

こんにちは、私はAIです。私はOpenAIが開発した自然言語処理モデルです。私の目的は、ユーザーが質問や要求をすると、最善の回答や応答を提供することです。私は様々なトピックについての情報を持っており、文法やスタイルの修正も行うことができます。どのようにお手伝いできますか？


### Prompts module

モデルへの入力を組み立てるmoduleであり、次の要素がある
- Prompt Templates
- Chat Prompt Templates
- Example Selectors
- Output Parsers

ここでは、Prompt Templatesについて説明する
- ChatGPTへのプロンプトについてテンプレートつまり例文を作成することができる



次のコードでは、commandがlsに置き換わる
- Promptの長さを考慮して埋め込む
- 出力する形式を指定して埋め込む

などが可能であり、単純なPythonコードによる埋め込みよりも高度な処理が可能である

In [ ]:
from langchain.prompts import PromptTemplate

template = """
次のコマンドの概要を説明してください。

コマンド: {command}
"""

prompt = PromptTemplate(
    input_variables=["command"],
    template=template,
)

result = prompt.format(command="ls")
print(result)


次のコマンドの概要を説明してください。

コマンド: ls



### Chains module

Models, Templates, Chainsなどのmoduleを連結する

なお、LangChainの挙動の詳細を確認するため、
`langchain.verbose = True`
としている
- いろいろと意味のない文章や宣伝も表示されるが、不要な場合は、Falseとするとよい

In [ ]:
import langchain
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

langchain.verbose = True

# Model を用意
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Prompt を用意
template = """
次のコマンドの概要を説明してください。

コマンド: {command}
"""
prompt = PromptTemplate(
    input_variables=["command"],
    template=template,
)

# Chain を作成
chain = LLMChain(llm=chat, prompt=prompt)

# 実行
result = chain.run("ls")
print(result)



> Entering new LLMChain chain...
Prompt after formatting:

次のコマンドの概要を説明してください。

コマンド: ls


> Finished chain.
lsコマンドは、UnixやUnix系のオペレーティングシステムで使用されるコマンドで、指定されたディレクトリ内のファイルやディレクトリの一覧を表示するために使用されます。デフォルトでは、現在のディレクトリの内容が表示されますが、オプションを使用して他のディレクトリの内容を表示することもできます。ファイルやディレクトリの名前、パーミッション、所有者、サイズなどの情報が表示されます。


chain.runを用いて、構築したcheinが順に実行される
- 最初に文字の埋め込み(prompt)が行なわれる
- 次にchat modelにより実際に通信が行なわれる

このChainには各種存在する

In [ ]:
#スロット待ち
if openai_wait:
  time.sleep(60)

#### SimpleSequentialChain

ChainとChainを直列に連結する



例えば、次の例を考えてみよう

簡単な算数の問題を問い合わせ、それに対して回答を得る場合、詳細な手順を聞くようにすると解答の精度が向上する

しかしながら、欲しいのは最終的な答えであって、途中経過は不要であるとする

すると、まず、詳細な手順を含む答えを得てから、その答えを要約して最後の答えだけ得るようにするとよい

つまり、ChatGPTを2回利用して最終的に欲しい回答を獲得することになる

In [ ]:
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

langchain.verbose = True

# Model を用意
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# 1 つ目の Prompt と Chain を用意
cot_template = """
以下の質問に回答してください。

### 質問 ###
{question}
### 質問終了 ###

ステップバイステップで考えましょう。
"""
cot_prompt = PromptTemplate(
    input_variables=["question"],
    template=cot_template,
)
cot_chain = LLMChain(llm=chat, prompt=cot_prompt)

# 2 つ目の Prompt と Chain を用意
summarize_template = """
入力を結論だけ抜き出して記述してください。

### 入力 ###
{input}
### 入力終了 ###
"""
summarize_prompt = PromptTemplate(
    input_variables=["input"],
    template=summarize_template,
)
summarize_chain = LLMChain(llm=chat, prompt=summarize_prompt)

# 2 つの Chain を直列に繋ぐ
cot_summarize_chain = SimpleSequentialChain(
    chains=[cot_chain, summarize_chain])

# 実行
result = cot_summarize_chain(
    "私は市場に行って10個のリンゴを買いました。隣人に2つ、修理工に2つ渡しました。それから5つのリンゴを買って1つ食べました。残りは何個ですか？")
print(result["output"])



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

以下の質問に回答してください。

### 質問 ###
私は市場に行って10個のリンゴを買いました。隣人に2つ、修理工に2つ渡しました。それから5つのリンゴを買って1つ食べました。残りは何個ですか？
### 質問終了 ###

ステップバイステップで考えましょう。


> Finished chain.
1. 最初に市場で10個のリンゴを買いました。
2. 隣人に2つのリンゴを渡しました。残りは8個です。
3. 修理工に2つのリンゴを渡しました。残りは6個です。
4. さらに5つのリンゴを買いました。残りは11個です。
5. 1つのリンゴを食べました。残りは10個です。

したがって、残りのリンゴは10個です。


> Entering new LLMChain chain...
Prompt after formatting:

入力を結論だけ抜き出して記述してください。

### 入力 ###
1. 最初に市場で10個のリンゴを買いました。
2. 隣人に2つのリンゴを渡しました。残りは8個です。
3. 修理工に2つのリンゴを渡しました。残りは6個です。
4. さらに5つのリンゴを買いました。残りは11個です。
5. 1つのリンゴを食べました。残りは10個です。

したがって、残りのリンゴは10個です。
### 入力終了 ###


> Finished chain.
残りのリンゴは10個です。

> Finished chain.
残りのリンゴは10個です。


なお、独自のpromptsやchains moduleを作成することも可能である

詳細は、LangChainのマニュアル https://python.langchain.com/docs/get_started/introduction を参照されたい


#### Output Persers

出力形式を指定するプロンプトの作成とPythonオブジェクトとのマッピングを提供する

<img src="http://class.west.sd.keio.ac.jp/dataai/text/chatapi2.jpg" width=700>

In [ ]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field, validator
from typing import List

langchain.verbose = True

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

class Recipe(BaseModel):
    ingredients: List[str] = Field(description="ingredients of the dish")
    steps: List[str] = Field(description="steps to make the dish")
    time: List[str] = Field(description="time to make the dish")
    sides: List[str] = Field(description="side menu of the dish")

template = """料理のレシピを教えてください。

{format_instructions}

料理名: {dish}
"""

parser = PydanticOutputParser(pydantic_object=Recipe)

prompt = PromptTemplate(
    template=template,
    input_variables=["dish"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain = LLMChain(llm=chat, prompt=prompt)

output = chain.run(dish="カレー")
print("=== output ===")
print(output)



> Entering new LLMChain chain...
Prompt after formatting:
料理のレシピを教えてください。

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"title": "Ingredients", "description": "ingredients of the dish", "type": "array", "items": {"type": "string"}}, "steps": {"title": "Steps", "description": "steps to make the dish", "type": "array", "items": {"type": "string"}}, "time": {"title": "Time", "description": "time to make the dish", "type": "array", "items": {"type": "string"}}, "sides": {"title": "Sides", "description": "side menu of the dish", "type": "array", "items": {"ty

最後に、recipe型にマッピングする

In [ ]:
recipe = parser.parse(output)
print("=== recipe object ===")
print(recipe)

=== recipe object ===
ingredients=['玉ねぎ', 'にんじん', 'じゃがいも', '豚肉', 'カレールー', '水'] steps=['玉ねぎ、にんじん、じゃがいもを切る', '豚肉を炒める', '野菜を加えて炒める', '水を加えて煮込む', 'カレールーを加えて溶かす', '煮込んで完成'] time=['30分'] sides=['ごはん', 'フライドチキン']


### Indexes

ChatGPTは学習に用いたデータセットの範疇でのみ答えを出すため、新しい知識や概念については、正しく解答することが難しい

例えば、次のような質問に対しては、お手上げになっている

<img src="http://class.west.sd.keio.ac.jp/dataai/text/langchain1.jpg" width=500>

ここで、質問に対する回答を得るうえで必要となる情報を渡してから処理させると、おおよそ正しい回答を得ることができるようになる

<img src="http://class.west.sd.keio.ac.jp/dataai/text/langchain2.jpg" width=500>


コンテキストとして、情報を加えることで正しい回答を与えることができており、これは強力な方法であるが、実際に用いる場合は注意が必要である

- ChatGPTには、入力文字数に制限があるため、長い文章を入力する必要がある場合はともかく、様々な情報を大量に与えておいて、そこから適切な回答を得るという利用は困難である
- 文字数、つまり入力トークン数も課金に関係するため、高コストとなる

まず、全ての情報を与えておいて、何かしら回答を得るということは非現実的であるが、自動化という点では有効な手段である



#### Vector Store

そこで、Vector Storeを活用する
- 文章をベクトル化してVector Storeに保存、入力と近しいベクトルの文章をVector Storeから検索してcontextに含める手法
- 全文章ではなく、「大事と思われる部分文章群についてのみ」情報を与える
  - これには、、embeddingと呼ばれる内部ベクトル表現への変換を行い、そのベクトルの近接性を用いて、どの文章が重要かを判断している
  - 単語をベクトル化するトークンではない点に注意すること

<img src="http://class.west.sd.keio.ac.jp/dataai/text/langchain3.jpg" width=700>



実際にindexを扱う

ここでは、LangChainのドキュメントを参考にして、質問できるようにする

最新のドキュメントはgitで公開されているため、LangChainのgitをcloneする

In [ ]:
!git clone https://github.com/hwchase17/langchain.git

Cloning into 'langchain'...
remote: Enumerating objects: 66945, done.
remote: Counting objects: 100% (7800/7800), done.
remote: Compressing objects: 100% (498/498), done.
remote: Total 66945 (delta 7499), reused 7396 (delta 7300), pack-reused 59145
Receiving objects: 100% (66945/66945), 70.74 MiB | 25.34 MiB/s, done.
Resolving deltas: 100% (48440/48440), done.


langchainのディレクトリに入る

In [ ]:
!cd langchain

以降の動作確認で必要となるライブラリを導入する

In [ ]:
!pip install --quiet unstructured tabulate pdf2image pytesseract chromadb tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.4/422.4 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 kB 44.1 MB/s e

まず、ディレクトリの中の全体を読むための便利なDirectoryLoaderを用いて、ある場所にあるファイルをサブディレクトリもまとめて取得する
- ここでは拡張子がmdであるファイルに限定している

- そこから次々に文章を取得して、VectorstoreIndexCreatorで、Vectorsoreに格納していく
  - この時、embeddingと呼ばれる内部ベクトル表現への変換も同時に行っている

In [ ]:
from langchain.document_loaders import DirectoryLoader
from langchain.indexes import VectorstoreIndexCreator

loader = DirectoryLoader("./langchain/docs/", glob="**/*.md")
index = VectorstoreIndexCreator().from_loaders([loader])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


では、そのvector storeを利用して、質問する

なお、現時点でChatGPTに"LangChain"について問い合わせると、データセットに含まれていないという回答になる

In [ ]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

result = index.query("LangChainにおけるIndex moduleについて概要を1文で説明してください。", llm=chat)
print(result)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Integrate with LangChain: if your product integrates with LangChain–or aspires to–we want to help make sure the experience is as smooth as possible for you and end users. Send us an email at hello@langchain.dev and tell us what you’re working on.
Become an Integration Maintainer: Partner with our team to ensure your integration stays up-to-date and talk directly with users (and answer their inquiries) in our Discord. Introduce yourself at hello@langchain.dev if you’d like to explore this role.

🌍 Meetups, Events, and Hackathons

global events calendar.

📣 Help Us Amplify Your Work

If you’re working on something you’re proud of, and think the LangChain community w

`langchain.verbose = True`のため、無意味な動作確認メッセージも表示されているが、要約文が出力されていることがわかる
- なお、若干回答は的を得ていない


DocumentLoadersは、webサイト、GoogleDrive、Slackなどを読み込む機能が存在しているため、様々な情報をVctor Storeに格納することができる

このように、ある特定分野のデータを一連の要素としてエンコードし、それぞれが内部で 1 つの「ベクトル」として表現している
- これには、単語であればWord2Vecなどが利用できるが、センテンスであるため、BERTをFine-TuningしたSentenceTransformerがある
- このベクトル数値は、多次元ベクトル空間で要素を相互に関連づけてマッピングしている

ベクトル要素がセマンティックであり、ある一つの意味を表していると考えるならば、そのベクトルの近接性が文脈関係の指標となりえる
- このベクトルはエンベディングと呼ばれる
- 互いに関連性のある意味要素がまとまって配置されるようにエンベディングを行う

特定分野の文脈によって、セマンティック要素は単語、フレーズ、センテンス、パラグラフ、文書全体、画像、あるいはまったく別のものになる可能性があり、エンベディングが最善というわけではない



プロンプトで必要となる文脈を生成するため、データベースに問い合わせを行い、ベクトル空間の入力と密接に関連する要素を抽出する必要がある

ベクトルデータストアは、大量のベクトルを保存し、問い合わせに答えるシステム
- 効率的な最近傍クエリアルゴリズム(k-NNなど)と適切なインデックスにより、データ検索を行う

#### SentenceTransformer

BERTのトークンごとのEmbeddingをPoolingすることで、テキストのEmbeddingを計算し、同じ意味のテキスト間のEmbeddingの距離を最小化するようにFine Tuningされている

次の図の左のようなSBERT構造を用いて学習を行い、例えばSNLI データセットを用いる
- なお、ここでは分類を目的関数としている
- 2つのBERTネットワークは、重みが同値である(シャムネットワーク構造)

推論時のSBERTでは、コサイン類似度を用いて2つの文章の類似度を求めるなどが可能となる

このuおよびvが、文章AとBのembeddingということになる

<img src="http://class.west.sd.keio.ac.jp/dataai/text/sentencetransformer.png" width=700>

OpenAIが提供するembeddingがSentenceTransformerを採用しているというわけではないが、このような文章を意味に基づいてベクトル化する技術は以前より構築されてきた




In [ ]:
#スロット待ち
if openai_wait:
  time.sleep(60)

#### OpenAIのEmbedding

では、実際にOpenAIのAPIを用いて文章のembeddingを行う
- 実際にはLangChainを用いていれば自動的にembeddingが行なわれるため、次に示すコードや仕組みが必要となることはない

In [ ]:
from openai.embeddings_utils import cosine_similarity
def text_to_embedding(text):
	response = openai.Embedding.create(
    	model= "text-embedding-ada-002",
    	input=[text]
	)
	return response["data"][0]["embedding"]

embeddingするテキストを準備する

In [ ]:
texts = ["吾輩は猫である", "私は猫です", "データサイエンスが好きです"]

準備したテキストのembeddingを求める

In [ ]:
import openai
embeddings = []
for text in texts:
  embeddings.append(text_to_embedding(text))

embeddingの次元を確認すると、1536次元とわかる

In [ ]:
len(embeddings[0])

1536

実際に値を確認する
- ほぼ意味はないが…

In [ ]:
embeddings[0]

[-0.0025837705470621586,
 -0.007356963120400906,
 -0.005908915773034096,
 -0.01717465929687023,
 -0.024922816082835197,
 0.013363677076995373,
 -0.012007119134068489,
 -0.0196353942155838,
 -0.002706807106733322,
 -0.023597804829478264,
 0.005460936110466719,
 0.01487797498703003,
 0.003249430563300848,
 -0.02796403132379055,
 -0.020443018525838852,
 -0.008353875949978828,
 0.034223128110170364,
 -0.00903530977666378,
 0.032658353447914124,
 -0.025200437754392624,
 0.030311191454529762,
 -0.002498591085895896,
 0.0018218894256278872,
 -0.013780108653008938,
 -0.027787363156676292,
 0.008820784278213978,
 0.007962682284414768,
 -0.028569750487804413,
 0.011262589134275913,
 -0.013300581835210323,
 0.006744934245944023,
 -0.011218422092497349,
 0.014562495984137058,
 -0.015508932061493397,
 -0.005473555065691471,
 -0.018663719296455383,
 -0.003760505933314562,
 0.004050746094435453,
 -0.002695765346288681,
 -0.00340086012147367,
 0.005470400210469961,
 0.0020048669539391994,
 -0.00256010

コサイン類似度を用いて類似度を求める
- OpenAIのライブラリが提供しているcosine_similarity関数を利用する

最初の2つの文章の類似度と後ろの2つの文章の類似度を求める

In [ ]:
cosine_similarity(embeddings[0], embeddings[1]), cosine_similarity(embeddings[1], embeddings[2])

(0.9513884610918777, 0.8207579882295654)

値から判断できるように、最初の2つの文章の類似度の方が高いといえる

### Memory

ChatGPTをブラウザで利用した場合、過去の会話の履歴を踏まえて返答するが、APIではそのような振る舞いは行わない

APIを利用する場合は、過去の履歴をプロンプトに入力する必要がある

実際にその振る舞いを確認する

次のような関数を用意してAPIを用いて文章を渡す

In [ ]:
def post_chat_completions(content):
  url = "https://api.openai.com/v1/chat/completions"
  headers = {
      "Content-Type": "application/json",
      "Authorization": "Bearer " + os.environ["OPENAI_API_KEY"]
  }
  data = {
      "model": "gpt-3.5-turbo",
      "messages": [
          {"role": "user", "content": content}
      ],
      "temperature": 0,
  }

  response = requests.post(url=url, headers=headers, json=data)
  print(json.dumps(response.json(), indent=2))

では、実際に名前を伝える

In [ ]:
post_chat_completions("Hi! I'm Keio Yukichi!")

{
  "id": "chatcmpl-7yzGp2HDSlmT28HyaAu7MKhl68fEV",
  "object": "chat.completion",
  "created": 1694769559,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello Keio Yukichi! How can I assist you today?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 16,
    "completion_tokens": 13,
    "total_tokens": 29
  }
}


"Hello Keio Yukichi! How can I assist you today?" といった回答が得られているであろう
- Generativeであるため、この答えは毎回異なる

その上で、名前を憶えているか聞いてみよう

In [ ]:
post_chat_completions("Do you know my name?")

{
  "id": "chatcmpl-7yzGqb7hdNxkhwKzMDECkAhRRE9Vg",
  "object": "chat.completion",
  "created": 1694769560,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "No, I do not know your name. As an AI language model, I don't have access to personal information unless you provide it to me."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 13,
    "completion_tokens": 30,
    "total_tokens": 43
  }
}


当然であるが、知らないという答えになる

そこで、過去の会話の履歴を全て入れて、同じ質問を行ってみよう

In [ ]:
post_chat_completions("""A: Hi! I'm Keio Yukichi!
B: Hello Keio Yukichi! How can I assist you today?
A: Do you know my name?
B: """)

{
  "id": "chatcmpl-7yzGsfPLXoKjtNpcT6NqQHYSABAKN",
  "object": "chat.completion",
  "created": 1694769562,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Yes, you introduced yourself as Keio Yukichi."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 44,
    "completion_tokens": 11,
    "total_tokens": 55
  }
}


となり、今度は"Yes, you introduced yourself as Keio Yukichi."と回答している

A:やB:は、会話しているのがどちらかを示す識別子であり、どのような形でもよい
- ただしLangChainは、内部でhumanとAIという用語を利用している

このように、会話の過去の履歴を含めて問い合わせを行うため、過去の履歴を記録し、挿入するという処理が必要となる
- これがMemory moduleの役割である

In [ ]:
#スロット待ち
if openai_wait:
  time.sleep(60)

実際に使うには、Chainの中にmemoryとしてConversationBufferMemory()を加えるだけでよい

プロンプト(文字を入力するための入力窓)が出てきたら、
- Hello. I'm Keio Yukichi
- Do you know my name?
- EOC
と入力する

EOCは会話を終了させるおまじないである
- これは、ChatGPTの機能ではなく、そのようにプログラムしている

In [ ]:
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory


langchain.verbose = True

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, max_tokens=100)
conversation = ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory()
)

while True:
    user_message = input("You: ")
    if(user_message == 'EOC'):
      break
    ai_message = conversation.predict(input=user_message)
    print(f"AI: {ai_message}")

You: Hello. I'm Keio Yukichi


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello. I'm Keio Yukichi
AI:

> Finished chain.
AI: Hello Keio Yukichi! How can I assist you today?
You: Do you know my name?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello. I'm Keio Yukichi
AI: Hello Keio Yukichi! How can I assist you today?
Human: Do you know my name?
AI:

> Finished chain.
AI: Yes, your name is Keio Yukichi.
You: EOC


roleには、system, assistant, userの3つがあることは述べた
- systemは設定に利用され、例えばChatGPTにキャラを与えるような命令も存在する

assistantとuserについて、assistant はAIからの回答、 user はユーザーからの発話であり、基本的にuserとして聞きたいことや、会話履歴を含めてを送ることになる
- この例では、userに、ユーザとChatGPTの両方の会話を入れ込んでいる

しかしながら、本来は、assistantがAIからの回答であることから、userにはユーザからの発話履歴のみ、assistantはAIからの発話履歴のみを入れるという形が望ましい
- これについては後で説明する

その他、様々なMemory moduleが提供されている
- ConversationBufferWindowMemory
  - ある範囲の会話履歴のみ入力する
- ConversationSummaryMemory
  - 会話履歴の要約を入力する

In [ ]:
#スロット待ち
if openai_wait:
  time.sleep(60)

### Agents

LLMが必要に応じて様々なタスクを実行すると便利と思うであろう

例えば、
- 検索エンジンで検索させる
- 実際にコマンドを実行させる
- プログラミング言語やスクリプト言語でコードを実行させる

これを行うのがAgents moduleである

Agentsの利用により、実際にはLLMが何かを操作するわけではないが、LLMが何かしらアプリを操作しているかのように動作させることができる

Agentsで操作可能なアプリの例
- bash (シェル)
- Google Search
- IFTTT WebHooks (スマートホーム等)
- Python REPL
- Requests (他のAPIを叩く)
- Wikipedia API

実際にAgentsを利用してみよう


In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI

langchain.verbose = True

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
tools = load_tools(["terminal"], llm=chat)
agent_chain = initialize_agent(
    tools, chat, agent="zero-shot-react-description")

result = agent_chain.run("What is your current directory?")
print(result)



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. You have access to the following tools:

terminal: Run shell commands on this Linux machine.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [terminal]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: What is your current directory?
Thought:

> Finished chain.
I can use the terminal to check the current directory.
Action: terminal
Action Input: pwd
Observation: /content

Thought:

> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. You have access to the foll

/usr/local/lib/python3.10/dist-packages/langchain/tools/shell/tool.py:32: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(



> Finished chain.
I now know the final answer
Final Answer: The current directory is /content.

> Finished chain.
The current directory is /content.


In [ ]:
#スロット待ち
if openai_wait:
  time.sleep(60)

単純に/content というディレクトリにいるという回答であるが、実際正解である

ただ、ここで疑問が生じる **なぜ、ChatGPTはこちらのシェル環境のカレントディレクトリがわかったのだろうか？**

これは、AgentsがMRKL(ミラクル)やReActなどを利用して動作しているためである
- MRKL(Multi-Round Knowledge Loop)
- ReAct (Reasoning/Acting) なお、Reactではない


ログをみると、LangChainは次のようなプロンプトを生成している
```
Answer the following questions as best you can. You have access to the following tools:

terminal: Run shell commands on this Linux machine.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [terminal]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
```

翻訳すると、
```
次の質問にできるだけ答えてください。あなたは以下のツールにアクセスできる：

terminal: このLinuxマシンでシェルコマンドを実行する。

以下の書式を使う：

Question: あなたが答えなければならない入力問題
Thought: 何をすべきかを常に考える。
Action: 取るべき行動。[terminal]のどれかであるべき。
Action Input: アクションへの入力
Observation: 行動の結果
...（このThought/Action/Action Input/ObservationはN回繰り返すことができる）
Thought: 最終的な答えがわかった
Final Answer: 元の入力された質問に対する最終的な答え

始める！
```

となっており、これらの書式を用いて処理が進む
```
Question: What is your current directory?
```
という問いかけに対して、
```
Thought:I can use the "pwd" command to find out the current directory.
Action: terminal
Action Input: pwd
```
とChatGPTが返答する

そこで、AgentはAction Inputに記載されているコマンドを実行する
- その結果を Observationとして埋め込む

さらに質問を続けるが、先のプロンプトに加えて、次の文章が加わっている
- つまり、これまでの動作をプロンプトに入力している

```
Question: What is your current directory?
Thought:I can use the "pwd" command to find out the current directory.
Action: terminal
Action Input: pwd
Observation: /content

Thought:
```
これに対して
```
I now know the final answer
Final Answer: The current directory is /content.
```
と回答している

Final Answerとして現在のディレクトリは/contentであることが示されており、Final Answerが返されたので、実行を終了している

では、どんどんやってみよう

In [ ]:
result = agent_chain.run("Make a new directory called 'testdir-by-agent'")
print(result)



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. You have access to the following tools:

terminal: Run shell commands on this Linux machine.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [terminal]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: Make a new directory called 'testdir-by-agent'
Thought:


message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
api_version=None data='{"messages": [{"role": "user", "content": "Answer the following questions as best you can. You have access to the following tools:\\n\\nterminal: Run shell commands on this Linux machine.\\n\\nUse the following format:\\n\\nQuestion: the input question you must answer\\nThought: you should always think about what to do\\nAction: the action to take, should be one of [terminal]\\nAction Input: the input to the action\\nObservation: the result of the action\\n... (this Thought/Action/Action Input/Observation can repeat N times)\\nThought: I now know the final answer\\nFinal Answer: the final answer to the original input question\\n\\nBegin!\\n\\nQuestion: Make a new directory called \'testdir-by-agent\'\\nThought:"}], "model": "gpt-3.5-turbo", "max_tokens": null, "stream": false, "n": 1, "temperature": 0.0, "stop": ["\\nObservation:", "\\n\\tObservation:"]}' message='Post det


> Finished chain.
I need to use the terminal to create a new directory.
Action: terminal
Action Input: mkdir testdir-by-agent
Observation: mkdir: cannot create directory ‘testdir-by-agent’: File exists

Thought:

> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. You have access to the following tools:

terminal: Run shell commands on this Linux machine.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [terminal]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: Make a new directory called 'testdir-by-agent'
Thought:I need to use the terminal to create a new directory.
Action: terminal
Action Input: mkdi

message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=1230 request_id=2887f91d67bf9e8320742c9a267d496d response_code=200
body='{\n  "id": "chatcmpl-7yzOkrxRjSKx04g9TLbNwmMqW6A3C",\n  "object": "chat.completion",\n  "created": 1694770050,\n  "model": "gpt-3.5-turbo-0613",\n  "choices": [\n    {\n      "index": 0,\n      "message": {\n        "role": "assistant",\n        "content": "The directory \'testdir-by-agent\' already exists.\\nFinal Answer: The directory \'testdir-by-agent\' already exists."\n      },\n      "finish_reason": "stop"\n    }\n  ],\n  "usage": {\n    "prompt_tokens": 189,\n    "completion_tokens": 25,\n    "total_tokens": 214\n  }\n}\n' headers='{\'Date\': \'Fri, 15 Sep 2023 09:27:31 GMT\', \'Content-Type\': \'application/json\', \'Transfer-Encoding\': \'chunked\', \'Connection\': \'keep-alive\', \'access-control-allow-origin\': \'*\', \'Cache-Control\': \'no-cache, must-revalidate\', \'openai-model\': \'gpt-3.5-turbo-0613\', \'

ファイルも作ってみよう

なお、途中で無料枠の場合はスロットを使い切ってしまうので、ワーニングメッセージと待ちが発生する

In [ ]:
result = agent_chain.run("Create new file called test.txt in the directory of testdir-by-agent and store the text of This is test in the file.")
print(result)

message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
api_version=None data='{"messages": [{"role": "user", "content": "Answer the following questions as best you can. You have access to the following tools:\\n\\nterminal: Run shell commands on this Linux machine.\\n\\nUse the following format:\\n\\nQuestion: the input question you must answer\\nThought: you should always think about what to do\\nAction: the action to take, should be one of [terminal]\\nAction Input: the input to the action\\nObservation: the result of the action\\n... (this Thought/Action/Action Input/Observation can repeat N times)\\nThought: I now know the final answer\\nFinal Answer: the final answer to the original input question\\n\\nBegin!\\n\\nQuestion: Create new file called test.txt in the directory of testdir-by-agent and store the text of This is test in the file.\\nThought:"}], "model": "gpt-3.5-turbo", "max_tokens": null, "stream": false, "n": 1, "temperature": 0.0, "



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. You have access to the following tools:

terminal: Run shell commands on this Linux machine.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [terminal]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: Create new file called test.txt in the directory of testdir-by-agent and store the text of This is test in the file.
Thought:


message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=1247 request_id=d954d0aa4c12d1a49896fd4d7dcd0539 response_code=200
body='{\n  "id": "chatcmpl-7yzOlnsOJ17droAajQJfcgXzGkIPs",\n  "object": "chat.completion",\n  "created": 1694770051,\n  "model": "gpt-3.5-turbo-0613",\n  "choices": [\n    {\n      "index": 0,\n      "message": {\n        "role": "assistant",\n        "content": "I need to create a new file and write some text into it.\\nAction: terminal\\nAction Input: touch testdir-by-agent/test.txt"\n      },\n      "finish_reason": "stop"\n    }\n  ],\n  "usage": {\n    "prompt_tokens": 161,\n    "completion_tokens": 28,\n    "total_tokens": 189\n  }\n}\n' headers='{\'Date\': \'Fri, 15 Sep 2023 09:27:32 GMT\', \'Content-Type\': \'application/json\', \'Transfer-Encoding\': \'chunked\', \'Connection\': \'keep-alive\', \'access-control-allow-origin\': \'*\', \'Cache-Control\': \'no-cache, must-revalidate\', \'openai-model\': \'gpt-3.5-turbo-0613


> Finished chain.
I need to create a new file and write some text into it.
Action: terminal
Action Input: touch testdir-by-agent/test.txt
Observation: 
Thought:

> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. You have access to the following tools:

terminal: Run shell commands on this Linux machine.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [terminal]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: Create new file called test.txt in the directory of testdir-by-agent and store the text of This is test in the file.
Thought:I need to create a new file and write some text into it.
Action: termi

message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
api_version=None data='{"messages": [{"role": "user", "content": "Answer the following questions as best you can. You have access to the following tools:\\n\\nterminal: Run shell commands on this Linux machine.\\n\\nUse the following format:\\n\\nQuestion: the input question you must answer\\nThought: you should always think about what to do\\nAction: the action to take, should be one of [terminal]\\nAction Input: the input to the action\\nObservation: the result of the action\\n... (this Thought/Action/Action Input/Observation can repeat N times)\\nThought: I now know the final answer\\nFinal Answer: the final answer to the original input question\\n\\nBegin!\\n\\nQuestion: Create new file called test.txt in the directory of testdir-by-agent and store the text of This is test in the file.\\nThought:I need to create a new file and write some text into it.\\nAction: terminal\\nAction Input: touch


> Finished chain.
I have created a new file called test.txt in the directory testdir-by-agent.
Action: terminal
Action Input: echo "This is a test" > testdir-by-agent/test.txt
Observation: 
Thought:

> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. You have access to the following tools:

terminal: Run shell commands on this Linux machine.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [terminal]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: Create new file called test.txt in the directory of testdir-by-agent and store the text of This is test in the file.
Thought:I need to create a new file and 

message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
api_version=None data='{"messages": [{"role": "user", "content": "Answer the following questions as best you can. You have access to the following tools:\\n\\nterminal: Run shell commands on this Linux machine.\\n\\nUse the following format:\\n\\nQuestion: the input question you must answer\\nThought: you should always think about what to do\\nAction: the action to take, should be one of [terminal]\\nAction Input: the input to the action\\nObservation: the result of the action\\n... (this Thought/Action/Action Input/Observation can repeat N times)\\nThought: I now know the final answer\\nFinal Answer: the final answer to the original input question\\n\\nBegin!\\n\\nQuestion: Create new file called test.txt in the directory of testdir-by-agent and store the text of This is test in the file.\\nThought:I need to create a new file and write some text into it.\\nAction: terminal\\nAction Input: touch


> Finished chain.
I have successfully written the text "This is a test" into the file test.txt.
Final Answer: The file test.txt has been created in the directory testdir-by-agent and it contains the text "This is a test".

> Finished chain.
The file test.txt has been created in the directory testdir-by-agent and it contains the text "This is a test".


message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=2105 request_id=a2f5e9b50c831a8d86b66f72b242839b response_code=200
body='{\n  "id": "chatcmpl-7yzPSPtn7WSW1Yky8Bh7Xa57xlbnv",\n  "object": "chat.completion",\n  "created": 1694770094,\n  "model": "gpt-3.5-turbo-0613",\n  "choices": [\n    {\n      "index": 0,\n      "message": {\n        "role": "assistant",\n        "content": "I have successfully written the text \\"This is a test\\" into the file test.txt.\\nFinal Answer: The file test.txt has been created in the directory testdir-by-agent and it contains the text \\"This is a test\\"."\n      },\n      "finish_reason": "stop"\n    }\n  ],\n  "usage": {\n    "prompt_tokens": 239,\n    "completion_tokens": 46,\n    "total_tokens": 285\n  }\n}\n' headers='{\'Date\': \'Fri, 15 Sep 2023 09:28:17 GMT\', \'Content-Type\': \'application/json\', \'Transfer-Encoding\': \'chunked\', \'Connection\': \'keep-alive\', \'access-control-allow-origin\': \'*\'

In [ ]:
#スロット待ち
if openai_wait:
  time.sleep(60)

testdir-by-agentの下にtest.txtがあり、その中身がThis is a testであることを確認しよう

### MRKL(Multi-Round Knowledge Loop)

例えば、「現在の日本の首相の年齢から現在のフランスの首相の年齢を引いたらいくつですか？」という問いに対して、ChatGPTは答えることができるか？

これを直接WebのChatGPTに問い合わせても答えることができない

しかしながら、APIでは答えることができる
- つまり、APIでChatGPTをアクセスすると、Webとは異なる仕組みでアクセスできるということ

では、その手順であるが、まず質問に答えるための手順を考える

- 最初に問い。「現在の日本の首相の年齢から現在のフランスの首相の年齢を引いたらいくつですか？」（Question）

- Googleで未知の情報を調べるために検索ワードを考える(Thought)

- 現在の日本とフランスの首相の年齢を知る必要がある
  - 検索ワードは「現在の日本の首相の年齢」(Action Input)
  - さらに検索ワードは「現在のフランス首相の年齢」(Action Input)

- 検索を実行「現在の日本の首相の年齢」（Action）
  - 結果は65歳でした（obsabation)

- 検索を実行します「現在のフランス首相の年齢」（Action）
  - 結果は61歳でした（obsabation)

- 日本の首相の年齢とフランスの首相の年齢の差分を計算する必要がある（Thought）

- 計算（Action）

- 結果は4でした(obsabation)

- 答えは4歳です (final)

このようにMRKLは、ChatGPTが情報をもとに次のアクションを考え、結果を評価し、次のアクションを考えるというプロセスを繰り返すことで回答精度を上げる方法論である

考察（Thought）、観察（Observation）、行動（Action）のサイクルを繰り返すことで、回答精度が向上する

よく言われる、ステップバイステップで考えるように指示すると正答率が上がるのと似ているが、Agent側で実際に実行して応答できるように工夫されている

### Prompt Coding

プロンプトコーディングはChatGPTの活用において必須となる技術である

ChatGPTから精度の高い回答を得るために、人間に質問するのと同様に、質問力が重要であり、その質問の仕方に関する研究が進められている

例えば、以下のように役割の指定や回答の形式を細かく設定することで、正答率を上げることができる
- 質問や回答が定型化されており、プログラムで文字列を処理することが容易になり、解析が可能となる

```
あなたは、英語の先生です。これから私の英語を英語教師として文法の誤りを訂正して下ださい。
回答のフォーマットは以下のようにします。
あなたの英語：{入力分}
訂正後の英文:{英文例}
文法の解説:{解説1000文字以内}
```

### 実際のプロンプト

先の年齢差を問う問題に答えさせる場合、次のようなプロンプトが想定される
- 内容は、シェルを実行するAgentの問い合わせと酷似する

```
Answer the following questions as best you can.
You have access to the following tools:\n\n

Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\n
Calculator: Useful for when you need to answer questions about math.\n\n

Use the following format:\n\n

Question: the input question you must answer\n
Thought: you should always think about what to do\n
Action: the action to take, should be one of [Search, Calculator]\n
Action Input: the input to the action\n
Observation: the result of the action\n
... (this Thought/Action/Action Input/Observation can repeat N times)\n
Thought: I now know the final answer\n
Final Answer: the final answer to the original input question\n\n

Begin!\n\n


Question: 現在の日本の首相の年齢から現在のフランスの首相の年齢を引いたらいくつですか？ 計算してください\nThought:')
```

最初の"Use the following format:\n\n" 以前について、

ここで、toolsについて何がどのように利用できるかを伝えているが、重要な点は次の通りである

- Searchというツール名
  - コロンの前にツール名が記載されている
- ユースケースを伝える
  - (時事問題に関する質問に答える必要があるときに便利です)
- 入力形式を指定する
  - (入力は、検索クエリである必要がある)

「入力は検索クエリである必要がある」と伝えているため、半角スペース区切りの単語単位での検索クエリを作成するようになる
- ChatGPTは時事問題に関する内容は、Searchツールを使うようになる

最初の"Use the following format:\n\n" 以降について

進め方とフォーマットを伝えている

- Question:質問内容を記載
- Thought:何をすべきかを常に考える必要がある
  - アクションを考えるように指示
- Action: 実行するアクションは、 [Search, Calculator]のいずれかである必要がある
  - アクション名はツール名と同じであり、ChatGPTからActionの指示が出る際には[Search,Calculator]のキーワードが出力される
- Action Input: アクションへの入力
  - Searchの場合は指示されたクエリ形式で入力する
- Observation:Actionの結果
  - アクションの結果を表示

さらに、最後について

- (this Thought/Action/Action Input/Observation can repeat N times)
  - N回繰り返すは、答えが出ない場合に打ち切る回数や、API利用料金を抑えるための制限回数として、Nを指定できるようにしている
- Thought:回答が判明したら下記に進みます
- Final Answer:最終的な回答をします

実際に試行すると次のような結果を得ることができる

```
> Entering new AgentExecutor chain...
I need to find out the age of the current Japanese and French Prime Ministers
Action: Search
Action Input: "age of current Japanese Prime Minister"params

Observation: 65歳
Thought: Now I need to find out the age of the current French Prime Minister
Action: Search
Action Input: "age of current French Prime Minister"params

Observation: 61歳
Thought: I now know the final answer
Final Answer: 4歳
```



##  Chat API におけるプロンプトの構築

先のmemoryの例に加えて、
`import openai`

および

`langchain.verbose = True`

を追加して、ログを詳細に取得する

プロンプトに対して、
- Hi, I'm Keio Yukichi.
- Do you know my name?
- EOC

と入力する


In [ ]:
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
import openai

langchain.verbose = True
openai.log = "debug"

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, max_tokens=100)
conversation = ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory()
)

while True:
    user_message = input("You: ")
    if(user_message == 'EOC'):
      break
    ai_message = conversation.predict(input=user_message)
    print(f"AI: {ai_message}")

You: Hi, I'm Keio Yukichi.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, I'm Keio Yukichi.
AI:


message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
api_version=None data='{"messages": [{"role": "user", "content": "The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\\n\\nCurrent conversation:\\n\\nHuman: Hi, I\'m Keio Yukichi.\\nAI:"}], "model": "gpt-3.5-turbo", "max_tokens": 100, "stream": false, "n": 1, "temperature": 0.0}' message='Post details'
message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=636 request_id=644ea4c95e6883c8ac31f6f774fdb9fd response_code=200
body='{\n  "id": "chatcmpl-7yzQchyZJ9Qq5yWxxhf7hHH9NDho1",\n  "object": "chat.completion",\n  "created": 1694770166,\n  "model": "gpt-3.5-turbo-0613",\n  "choices": [\n    {\n      "index": 0,\n      "message": {\n        "role": "assistant",\n        "content": "Hel


> Finished chain.
AI: Hello Keio Yukichi! How can I assist you today?
You: Do you know my name?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, I'm Keio Yukichi.
AI: Hello Keio Yukichi! How can I assist you today?
Human: Do you know my name?
AI:


message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
api_version=None data='{"messages": [{"role": "user", "content": "The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\\n\\nCurrent conversation:\\nHuman: Hi, I\'m Keio Yukichi.\\nAI: Hello Keio Yukichi! How can I assist you today?\\nHuman: Do you know my name?\\nAI:"}], "model": "gpt-3.5-turbo", "max_tokens": 100, "stream": false, "n": 1, "temperature": 0.0}' message='Post details'
message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=950 request_id=a1062ddf513a883c5bdc92d9744d5941 response_code=200
body='{\n  "id": "chatcmpl-7yzQlKc7LGXLeCdoGVG4ppVT0i97F",\n  "object": "chat.completion",\n  "created": 1694770175,\n  "model": "gpt-3.5-turbo-0613",\n  "choices": [\n    {\n      "inde


> Finished chain.
AI: Yes, your name is Keio Yukichi. You just mentioned it in your previous message.
You: EOC


ログを参照することで、動作の詳細を獲得できる

例えば、Memoryにより過去の履歴をプロンプトを与えることができるが、具体的には次のような動作をしている

```
api_version=None data='{"messages": [{"role": "user", "content": "The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\\n\\nCurrent conversation:\\nHuman: Hi, I\'m Keio Yukichi.\\nAI: Hello Keio Yukichi! How can I assist you today?\\nHuman: Do you know my name?\\nAI:"}], "model": "gpt-3.5-turbo", "max_tokens": 100, "stream": false, "n": 1, "temperature": 0.0}' message='Post details'
```

このように、すべてuserメッセージとして混入している

本来は、ChatGPTの言葉は、assistantとして入力するべきであろうことがわかる

そこで、これを使い分けるには、次のようにする

SystemMessage, HumanMessage、またAIMessageを用いて、それぞれの会話を仕分けできる

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, max_tokens=100)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi! I'm Keio Yukichi!"),
    AIMessage(content="Yes, You are Keio Yukichi.")
]

result = chat(messages)
print(result)

message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
api_version=None data='{"messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "Hi! I\'m Keio Yukichi!"}, {"role": "assistant", "content": "Yes, You are Keio Yukichi."}], "model": "gpt-3.5-turbo", "max_tokens": 100, "stream": false, "n": 1, "temperature": 0.0}' message='Post details'


content='Hello Keio Yukichi! How can I assist you today?' additional_kwargs={} example=False


message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=637 request_id=052f35a55c3bed8aa9180f7b6db726a6 response_code=200
body='{\n  "id": "chatcmpl-7yzQs3Rewykru3OhxVax2fFBXcSet",\n  "object": "chat.completion",\n  "created": 1694770182,\n  "model": "gpt-3.5-turbo-0613",\n  "choices": [\n    {\n      "index": 0,\n      "message": {\n        "role": "assistant",\n        "content": "Hello Keio Yukichi! How can I assist you today?"\n      },\n      "finish_reason": "stop"\n    }\n  ],\n  "usage": {\n    "prompt_tokens": 39,\n    "completion_tokens": 13,\n    "total_tokens": 52\n  }\n}\n' headers='{\'Date\': \'Fri, 15 Sep 2023 09:29:42 GMT\', \'Content-Type\': \'application/json\', \'Transfer-Encoding\': \'chunked\', \'Connection\': \'keep-alive\', \'access-control-allow-origin\': \'*\', \'Cache-Control\': \'no-cache, must-revalidate\', \'openai-model\': \'gpt-3.5-turbo-0613\', \'openai-organization\': \'user-3ayltofgziignqdsvkwyx6sh\', \'openai-proces

In [ ]:
#スロット待ち
if openai_wait:
  time.sleep(60)

これを踏まえて、先ほどのループ問い合わせプログラムを改善する

```
    memory.chat_memory.add_user_message(user_message)
    ai_message = chat(memory.chat_memory.messages)
    memory.chat_memory.add_ai_message(ai_message.content)
```
とすることで、memoryに対してだれの発言かを仕分けして登録するようにする

実際に実行して、次のようにプロンプトに入力する
- Hi. I'm Keio Yukichi.
- Do you know my name?
- EOC

ログを見てみると、Do you know my name? の問い合わせの後、

`"messages": [{"role": "user", "content": "Hi. I\'m Keio Yukichi."}`とuserが入力した後、`{"role": "assistant", "content": "Hello Keio Yukichi! How can I assist you today?"}'とassistantが返答、さらに`{"role": "user", "content": "Do you know my name?"}`とuserが入力といった具合に、正しく仕分けされている


In [ ]:
langchain.verbose = True

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
memory = ConversationBufferMemory()

while True:
    user_message = input("You: ")
    if(user_message == 'EOC'):
      break
    memory.chat_memory.add_user_message(user_message)
    ai_message = chat(memory.chat_memory.messages)
    memory.chat_memory.add_ai_message(ai_message.content)
    print(f"AI: {ai_message.content}")


You: Hi. I'm Keio Yukichi.


message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
api_version=None data='{"messages": [{"role": "user", "content": "Hi. I\'m Keio Yukichi."}], "model": "gpt-3.5-turbo", "max_tokens": null, "stream": false, "n": 1, "temperature": 0.0}' message='Post details'
message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=693 request_id=b416770493eb1ed78edf090cfcb4fe79 response_code=200
body='{\n  "id": "chatcmpl-7yzTJWyWuMfRVELqrLOiEnUR9NUrr",\n  "object": "chat.completion",\n  "created": 1694770333,\n  "model": "gpt-3.5-turbo-0613",\n  "choices": [\n    {\n      "index": 0,\n      "message": {\n        "role": "assistant",\n        "content": "Hello Keio Yukichi! How can I assist you today?"\n      },\n      "finish_reason": "stop"\n    }\n  ],\n  "usage": {\n    "prompt_tokens": 16,\n    "completion_tokens": 13,\n    "total_tokens": 29\n  }\n}\n' headers='{\'Date\': \'Fri, 15 Sep 2023 09:32:14 GMT\', \'Content-Type\

AI: Hello Keio Yukichi! How can I assist you today?
You: Do you know my name?


message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
api_version=None data='{"messages": [{"role": "user", "content": "Hi. I\'m Keio Yukichi."}, {"role": "assistant", "content": "Hello Keio Yukichi! How can I assist you today?"}, {"role": "user", "content": "Do you know my name?"}], "model": "gpt-3.5-turbo", "max_tokens": null, "stream": false, "n": 1, "temperature": 0.0}' message='Post details'
message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=1011 request_id=e078f88bbd8cdc4cc465d6bd3f1b6f6e response_code=200
body='{\n  "id": "chatcmpl-7yzTR4PlojDnYJ7atOoAPz9c9ZglK",\n  "object": "chat.completion",\n  "created": 1694770341,\n  "model": "gpt-3.5-turbo-0613",\n  "choices": [\n    {\n      "index": 0,\n      "message": {\n        "role": "assistant",\n        "content": "Yes, you mentioned your name as Keio Yukichi. How can I help you, Keio Yukichi?"\n      },\n      "finish_reason": "stop"\n    }\n  ],\n  "

AI: Yes, you mentioned your name as Keio Yukichi. How can I help you, Keio Yukichi?
You: EOC


In [ ]:
#スロット待ち
if openai_wait:
  time.sleep(60)

# Gradioについて

Webアプリを簡単に実装できるPythonライブラリ

百聞は一見に如かずということで、早速実行してみよう

## gradio のインストール方法

```
pip install gradio
```

とし、例えば、

```
import gradio as gr
```
とすることで利用可能となる

In [ ]:
!pip -q install gradio

シンプルなWeb UIを作成して起動させる例を示す
- 名前を入力し、名前へのあいさつを出力するWeb UIである

Colab上で実行すると、Colabのwebに統合される
- URLが出力されている通り、そのURLにアクセスできる環境にあれば、webのページとして表示される
- この例では、`https://localhost:7860/`などと表示されているが、クリックすると実は`https://wq0lbccui9-496ff2e9c6d22116-7860-colab.googleusercontent.com/`に転送されており、ネットワークセキュリティ上隔離されたColabの外からアクセスできるようになる
- Colab環境では、セキュリティ上の問題もあり、このようなグローバルアドレスが提供されない場合は、別途ボートフォーワーディングなどの知識が必要な場合がある
- また、ローカル上で他のWeb UIアプリを動作させているなどにより、ポートが競合する可能性がある
  - この場合、7860 が 7861 や 7862 といった番号に変わることがある


また、gradio clientを用いることで、作成したwebアプリケーションをweb APIのように利用することもできる
- 下に小さく「Use via API」と記載されているが、これをクリックし、記載の通りに実行すると動作がわかるであろう
·

In [ ]:
import gradio as gr

# あいさつの関数
def greet(name):
    return "Hello " + name + "!"

# Interfaceの作成
demo = gr.Interface(
    fn=greet,
    inputs="text",
    outputs="text"
)

# 起動
demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

## 設計手順

次の手順となる
- コールバック関数を定義
- レイアウトを定義
- WebUIの起動

それぞれについて概要をみてみよう


### コールバック関数

まず、コールバック関数として、画面レイアウトでボタンが押された時に呼び出したい関数を記述する  

例えば、名前(text)、表示フラグ(boolean)、値(0から100の値)の3つを受け取る関数として次のような関数を想定する  
```
def my_func(my_name, is_disp, my_value):
    return f'### {my_name} ###', my_value * 100
```



### Interface

「Interface」は、関数をUIでラップするためのクラスであり、主なパラメータは次のとおり
- fn : ラップする関数
- inputs : 入力コンポーネント ("text"、"image"、"audio"など)
- outputs : 出力コンポーネント ("text"、"image"、"label"など)

画面レイアウトの作成として、画面に表示したいUIパーツと、ボタンなどが押された時のアクション(呼ぶ出したいコールバック関数の名前)を記述する  
- 入力UIとして、my_nameは"text"であり、is_dispは"checkbox"であり、my_valueは、0から100の値をスライダーで入力させるとすると、gr.Slider(0, 100)となり、上からこの順に入力UIがレイアウトされる
- 出力UIとして、ここでは、文字と数字を想定する

最終的に、次のような関数定義となる  
```
demo = gr.Interface(
    fn = my_func,
    inputs=["text", "checkbox", gr.Slider(0, 100)],
    outputs=["text", "number"],
)
```

なお、"text"に対して、より詳細な情報を与えることもできる

```
demo = gr.Interface(
    fn=my_func,
    inputs=[
      gr.Textbox(
        lines=2,  # 行数
        placeholder="Name Here..."  # プレースホルダ
      ),
      "checkbox", gr.Slider(0, 100)
    ]
    outputs=["text", "number"],
)

```


- 最後にWebUIを起動する  
シンプルに、  
```
demo.launch()
```
とするだけでよい

inputs には「クリアボタン」と「送信」ボタンが、outputs には「フラグする」ボタンが自動で追加される(フラグボタンは出力をローカルファイルに保存する)

簡易的なWebサーバが起動し、ブラウザ上でWeb UIが表示される

<img src="http://class.west.sd.keio.ac.jp/dataai/text/gradio1.jpg" width=700>


## より実践的な例

他にも様々な機能があるため、調べてみるとよい

```
import gradio as gr

# コールバック関数の定義
def callback_func(val1,val2,val3):
    return str(int(val1) * int(val2)), f"気温は {val3} 度です"

# 画面レイアウトの定義(Interfaceを使用)
app = gr.Interface(
    title="計算機",
    fn=callback_func,
    inputs=[
        gr.Textbox(label="入力欄1",lines=3, placeholder="ここに数値を入れてください..."),
        gr.Textbox(label="入力欄2",lines=5, placeholder="ここに数値を入れてください..."),
        gr.Slider(label="温度",minimum=0,maximum=100,step=1)
    ],
    outputs=[
        gr.Label(label="計算結果1",lines=3),
        gr.Textbox(label="計算結果2",lines=3)
    ]
    )

# Web UIの起動
app.launch(inbrowser=True)
```




### Blocks

簡易的に使用する場合はInterfaceを使い、より複雑なレイアウトを作る場合はBlocksを使う

- Interfaceのメリット

  - ショートカット文字列の使用が可能(Blocksでは使用不可)
  - Interfaceでは基本的なボタンを自動生成

- Blocksのメリット

  - Blocksでのみ利用可能なレイアウトがある
  - レイアウトが複雑になった場合でもwith文で可読性の高いコードが書ける

Blocksではレイアウトを指定でき、次のようなレイアウトがある



#### コンポーネントを横や縦に並べる  
gr.Row()やgr.Column()を利用する

コード中にコメントがあるので、切り替えてみるとよい

In [ ]:
import gradio as gr
def greet(name): return "Hello " + name + "!"
with gr.Blocks() as app:
  with gr.Row():
#  with gr.Column(scale=2):
    inputs = gr.Textbox(placeholder="名前を入力してね!", label="名前")
    outputs = gr.Textbox(label="挨拶")
    btn = gr.Button("クリックしてね!")
app.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

#### タブで表示する  


In [ ]:
import gradio as gr
def greet(name): return "Hello " + name + "!"
with gr.Blocks() as app: # 入力タブを定義
  with gr.Tab("入力タブ"):
    inputs = gr.Textbox(placeholder="名前を入力してね!", label="名前")
    btn = gr.Button("クリックしてね!") # 出力タブを定義
  with gr.Tab("出力タブ"):
    outputs = gr.Textbox(label="挨拶")
    btn.click(fn=greet, inputs=inputs, outputs=outputs)
app.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

#### 丸角の子要素を設ける

角が丸く、周囲にパディングがあるボックスである

In [ ]:
import gradio as gr
def greet(name): return "Hello " + name + "!"
with gr.Blocks() as app: # Box()関数でレイアウトを定義
  with gr.Box():
    inputs = gr.Textbox(placeholder="名前を入力してね!", label="名前")
    outputs = gr.Textbox(label="挨拶")
    btn = gr.Button("クリックしてね!") # イベントを定義
    btn.click(fn=greet, inputs=inputs, outputs=outputs)
app.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

#### 子要素を折りたたみ可能にする

Accordionは、子要素を折りたたみ可能なセクションに配置する

openパラメータにより初期状態で開いているか(True)、閉じているか(False)を指定でき、デフォルトはOpen(True)


In [ ]:
import gradio as gr
def greet(name): return "Hello " + name + "!"
with gr.Blocks() as app: # Accordion()関数でレイアウトを定義
  with gr.Accordion(label="アプリを見る", open=False):
    inputs = gr.Textbox(placeholder="名前を入力してね!", label="名前")
    outputs = gr.Textbox(label="挨拶")
    btn = gr.Button("クリックしてね!") # イベントを定義
    btn.click(fn=greet, inputs=inputs, outputs=outputs)
app.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

### チャットを実装する

なんでもなく、"How are you?", "I know you", "I'm very hungry"のどれかをランダムに答えるアプリである

In [ ]:
import gradio as gr
import random
import time

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, chat_history):
        bot_message = random.choice(["How are you?", "I know you", "I'm very hungry"])
        chat_history.append((message, bot_message))
        time.sleep(2)
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

## LLMチャットの実装



In [ ]:
import langchain
from langchain.chat_models import ChatOpenAI

langchain.verbose = True

def chat(message: str) -> str:
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
    return llm.predict(message)


実際に試してみよう

- 私の名前は出田です。
- 私の名前がわかりますか？

と入力すると、
「いいえ、私はあなたの名前を知りません。」
といった回答になる

In [ ]:
import gradio as gr

def respond(message, chat_history):
    bot_message = chat(message)
    chat_history.append((message, bot_message))
    return "", chat_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

次のコードここから連続して実行するとエラーになる
- 上記のコードがエラーになるため、まず実行を留める
- 確認が終わったら、その先のコードを待ってから実行すること

In [ ]:
from google.colab import runtime
runtime.unassign()

In [ ]:
!pip install --quiet langchain openai gradio

In [ ]:
from dotenv import load_dotenv
load_dotenv(verbose=True)

True

In [ ]:
import langchain
from langchain.chat_models import ChatOpenAI

langchain.verbose = True

def chat(message: str) -> str:
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
    return llm.predict(message)

次に、過去の会話履歴を踏まえて回答するように chat関数を次のように更新する


In [ ]:
import langchain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ChatMessageHistory
from langchain.schema import HumanMessage

langchain.verbose = True

def chat(message: str, history: ChatMessageHistory) -> str:
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

    messages = history.messages
    messages.append(HumanMessage(content=message))

    return llm(messages).content

実際に試してみよう

- 私の名前は出田です。
- 私の名前がわかりますか？

と入力すると、
「はい、先ほど出田さんとおっしゃいましたよね。」
といった回答になる

In [ ]:
import gradio as gr
from langchain.memory import ChatMessageHistory

def respond(message, chat_history):
    history = ChatMessageHistory()
    for [user_message, ai_message] in chat_history:
        history.add_user_message(user_message)
        history.add_ai_message(ai_message)

    bot_message = chat(message, history)
    chat_history.append((message, bot_message))
    return "", chat_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

# 課題

履歴付きチャットボットを改造し、常に回答が子供の対応であるようにプロンプトエンジニアリングを行いなさい

単純に、入力問い合わせに対して、「考え方や言葉遣いを6歳の子供のようにして答えなさい」といった言葉を付け加えなさい